In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools

import matplotlib.pyplot as plt
import numpy as np

from scipy import stats
from code.python.mpmath_integration import quad_phi

from mpmath import mp

from code.nig import nig_cdf

In [3]:
alpha = 10
beta = 0.0
mu = 2
delta = 1
x = 1
# x, mu

# x, alpha, mu, delta = (3.2770259382044173, 24.10015435697935, -0.9561424909702456, 46.56789071292953)

x = -1.8816854798951448
alpha = 23.184287219731825
mu = -0.6779473810531575
delta = 46.86837947038307

x, alpha, mu, delta = (0.1182162470025671,
 27.116782747705887,
 -2.1582741889630075,
 5.764235016147439)

# x, alpha, mu, delta = (-1.9103180092440883, 22.41626678065764, 2.9491781940248964, 0.14817937115534)
x = -1.9103180092440883
alpha = 22.41626678065764
mu = 2.9491781940248964
delta = 0.14817937115534

# x = 4.39
# alpha = 4.77
# beta = -0.41
# delta = 2.55
# mu = 0.57

# (-4.393734512267285,
#  4.767920679306739,
#  -0.4131443982576754,
#  2.5475082474089774,
#  0.5737187613352117)

omega = np.sqrt(delta ** 2 + (x-mu)**2)
xmu = x - mu
xmu2 = xmu ** 2
aow = alpha / omega
aw = alpha * omega
alpha * delta, alpha * omega, ((xmu <= 20) & (aow <= 0.25)) | ((xmu2 <= 1.25) & (aw <= 750)), x - mu

(3.3216283152081867, 108.9823943256035, False, -4.859496203268985)

In [4]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=300)

In [5]:
scipy_result = stats.norminvgauss.cdf(x=x, a=alpha*delta, b=beta*delta, loc=mu, scale=delta)

In [6]:
gnp_result = nig_cdf(x, alpha, beta, mu, delta)

integration
2.8842e-48
1 1
level 0 j = 1
level 1 j = 2
level 2 j = 4
level 3 j = 8
level 4 j = 16
level 5 j = 32
level 6 j = 63
level 7 j = 126
level 8 j = 250
level 9 j = 498


In [7]:
float(mp_result)

1.493232088943462e-49

In [8]:
gnp_result

1.4932320889434717e-49

In [9]:
mp.nprint(abs(mp_result / scipy_result - 1), 4)

2.911e-8


In [10]:
mp.nprint(abs(mp_result / gnp_result - 1), 4)

6.55e-15


### Test

In [ ]:
xxs = np.linspace(-10, 10, 20)

alphas = np.linspace(1e-3, 100, 100)
rab = np.linspace(1e-3, 1-1e-3, 50)
betas = np.concatenate((alphas[:50] * rab, -alphas[50:] * rab))
mus = np.linspace(-10, 10, 20)
deltas = np.linspace(1e-3, 50, 20)

A = np.c_[alphas, betas]
B = np.array(list(itertools.product(mus, xxs, deltas)))

#### 0. Test general

In [11]:
rs = np.random.default_rng(seed=1)

n_samples = 5000

xs = rs.uniform(-5, 5, n_samples)
alphas = rs.uniform(0.001, 5, n_samples)
r = rs.uniform(-0.999, 0.999, n_samples)
betas = alphas * r
mus = rs.uniform(-5, 5, n_samples)
deltas = rs.uniform(0.001, 5, n_samples)

In [13]:
(alphas > np.absolute(betas)).all()

True

In [14]:
with open('instances_general_small.csv', 'a') as f:
    header = 'x,alpha,beta,mu,delta,mpmath,scipy,gnp\n'
    f.write(header)

iter = 0
for x, alpha, beta, mu, delta in zip(xs, alphas, betas, mus, deltas):
    print(f'{iter} |  x={x} alpha={alpha} beta={beta} mu={mu} delta={delta}')
    iter += 1
    
    # Mpmath reference
    mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
    
    # Scipy
    scipy_result = stats.norminvgauss.cdf(x=x, a=alpha*delta, b=beta*delta, loc=mu, scale=delta)

    # GNP
    gnp_result = nig_cdf(x, alpha, beta, mu, delta)

    instance = f'{x},{alpha},{beta},{mu},{delta},{float(mp_result):.16E},{scipy_result:.16E},{gnp_result:.16E}\n'
    
    with open('instances_general_small.csv', 'a') as f:
        f.write(instance)

0 |  x=0.11821624700256717 alpha=0.20910814319293725 beta=0.03013405866485997 mu=3.7228768002892725 delta=1.801323334764928
1 |  x=4.504636963259353 alpha=3.6670393210851486 beta=-3.434033895784077 mu=-0.14710699554596474 delta=2.335966302763447
2 |  x=-3.5584038728036624 alpha=0.45025169066801013 beta=-0.31909313401396866 mu=2.7529723112178726 delta=2.2188962676431085
3 |  x=4.486494471372438 alpha=0.5452002968715581 beta=0.3533400411838956 mu=-0.04118369429112256 delta=1.5721315004435823
4 |  x=-1.8816854798951455 alpha=3.1862101280717035 beta=-1.0308119447800823 mu=-4.584931102089925 delta=0.4600106661041034
5 |  x=-0.766735510274243 alpha=4.23006627369601 beta=1.5624834357274597 mu=4.309375930016909 delta=4.871647122240735
6 |  x=3.2770259382044173 alpha=1.290082968637745 beta=-0.41922691903032716 mu=2.533817258543544 delta=1.1003177972096192
7 |  x=-0.908008636308387 alpha=2.531449324279798 beta=-2.1469274546452106 mu=-1.6908977003529646 delta=0.8842213938772882
8 |  x=0.495936876

In [15]:
df_test = pd.read_csv('instances_general_small.csv')
df_test

,x,alpha,beta,mu,delta,mpmath,scipy,gnp
0,0.118216,0.209108,0.030134,3.722877,1.801323,0.060465,0.060465,0.060465
1,4.504637,3.667039,-3.434034,-0.147107,2.335966,1.000000,1.000000,1.000000
2,-3.558404,0.450252,-0.319093,2.752972,2.218896,0.103112,0.103112,0.103112
3,4.486494,0.545200,0.353340,-0.041184,1.572132,0.913257,0.913257,0.913257
4,-1.881685,3.186210,-1.030812,-4.584931,0.460011,0.999999,0.999999,0.999999
...,...,...,...,...,...,...,...,...
4995,-1.513796,2.254001,0.034276,4.972005,4.470447,0.000022,0.000022,0.000022
4996,0.769818,3.157949,2.069574,1.386358,0.231704,0.003065,0.003065,0.003065
4997,2.510713,0.583478,0.478094,-0.828377,0.913498,0.879942,0.879942,0.879942
4998,-0.305063,1.599725,-0.655449,4.459278,0.525627,0.000499,0.000499,0.000499


In [16]:
with open('instances_general_algo.csv', 'a') as f:
    header = 'x,alpha,beta,mu,delta,mpmath,scipy,gnp\n'
    f.write(header)


for i, row in df_test.iterrows():
    x, alpha, beta, mu, delta, mp_result, scipy_result = row[['x', 'alpha', 'beta', 'mu', 'delta', 'mpmath', 'scipy']]

    # GNP
    gnp_result = nig_cdf(x, alpha, beta, mu, delta)
    print(x, alpha, beta, mu, delta, gnp_result)
    
    instance = f'{x},{alpha},{beta},{mu},{delta},{mp_result},{scipy_result},{gnp_result:.16E}\n'
    
    with open('instances_general_algo.csv', 'a') as f:
        f.write(instance)

0.00291221
1184 1442
level 0 j = 1
level 1 j = 2
level 2 j = 4
level 3 j = 8
level 4 j = 16
level 5 j = 32
level 6 j = 63
level 7 j = 126
0.1182162470025671 0.2091081431929372 0.0301340586648599 3.722876800289273 1.801323334764928 0.060465287651329656
level 8 j = 250
0.549727
37 37
level 0 j = 1
level 1 j = 2
level 2 j = 4
level 3 j = 8
level 4 j = 16
level 5 j = 32
level 6 j = 63
level 7 j = 126
level 8 j = 250
4.504636963259353 3.6670393210851486 -3.434033895784077 -0.1471069955459647 2.335966302763447 0.9999999999999952
0.00453697
521 622
level 0 j = 1
level 1 j = 2
level 2 j = 4
level 3 j = 8
level 4 j = 16
level 5 j = 32
level 6 j = 63
level 7 j = 126
level 8 j = 250
-3.5584038728036624 0.4502516906680101 -0.3190931340139686 2.752972311217873 2.2188962676431085 0.10311188458350642
-nan
304 -2147483648
level 0 j = 1
level 1 j = 2
level 2 j = 4
level 3 j = 8
level 4 j = 16
level 5 j = 32
level 6 j = 63
level 7 j = 126
level 8 j = 250
4.486494471372438 0.5452002968715581 0.3533400411

#### 1. Test x = $\mu$

In [ ]:
rs = np.random.default_rng(seed=1)

alphas = rs.uniform(0.001, 100, 1000)
r = rs.uniform(0.001, 0.999, 1000)
betas = alphas * r
deltas = rs.uniform(0.001, 100, 1000)

rs = np.random.default_rng(seed=1)

alphas = rs.uniform(2, 50, 500)
r = rs.uniform(-0.999, 0.999, 500)
betas = alphas * r
deltas = rs.uniform(2, 50, 500)

In [ ]:
with open('instances_x_eq_mu_large_series.csv', 'a') as f:
    header = 'alpha,beta,delta,mpmath,scipy,gnp\n'
    f.write(header)

iter = 0
for alpha, beta, delta in zip(alphas, betas, deltas):
    print(f'{iter} | alpha={alpha} beta={beta} delta={delta}')
    iter += 1
    x = mu = 0
    
    # Mpmath reference
    mp_result = quad_phi(0, alpha, beta, mu, delta, digits=100)
    
    # Scipy
    scipy_result = stats.norminvgauss.cdf(x=x, a=alpha*delta, b=beta*delta, loc=mu, scale=delta)

    # GNP
    gnp_result = nig_cdf(x, alpha, beta, mu, delta)

    instance = f'{alpha},{beta},{delta},{float(mp_result):.16E},{scipy_result:.16E},{gnp_result:.16E}\n'
    
    with open('instances_x_eq_mu_large_series.csv', 'a') as f:
        f.write(instance)

In [ ]:
iter = 0

with open('instances_x_eq_mu.csv', 'a') as f:
    header = 'alpha,beta,delta,mpmath,scipy,gnp\n'
    f.write(header)

for i in range(A.shape[0]):
    alpha, beta = A[i, :]
    for j in range(B.shape[0]):
        iter += 1
        mu, x, delta = B[j, :]

        print(f'{iter} | x={x} alpha={alpha} beta={beta} mu={mu} delta={delta}')
        # Mpmath reference
        
        mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
        
        # Scipy
        scipy_result = stats.norminvgauss.cdf(x=x, a=alpha*delta, b=beta*delta, loc=mu, scale=delta)

        # GNP
        gnp_result = nig_cdf(x, alpha, beta, mu, delta)

        instance = f'{x},{alpha},{beta},{mu},{delta},{float(mp_result):.16E},{scipy_result:.16E},{gnp_result:.16E}\n'
        
        with open('instances.csv', 'a') as f:
            f.write(instance)

#### 2. Test $\beta = 0$

In [ ]:
rs = np.random.default_rng(seed=1)

xs = rs.uniform(-10, 10, 1000)
alphas = rs.uniform(1e-3, 50, 1000)
mus = rs.uniform(-10, 10, 1000)
deltas = rs.uniform(1e-3, 50, 1000)

In [ ]:
with open('instances_beta_eq_zero.csv', 'a') as f:
    header = 'x,alpha,mu,delta,mpmath,scipy,gnp\n'
    f.write(header)

iter = 0
for x, alpha, mu, delta in zip(xs, alphas, mus, deltas):
    print(f'{iter} | x={x} alpha={alpha} mu={mu} delta={delta}')
    iter += 1
    
    # Mpmath reference
    mp_result = quad_phi(x, alpha, 0, mu, delta, digits=100)
    
    # Scipy
    scipy_result = stats.norminvgauss.cdf(x=x, a=alpha*delta, b=0, loc=mu, scale=delta)

    # GNP
    gnp_result = nig_cdf(x, alpha, 0, mu, delta)

    instance = f'{x},{alpha},{mu},{delta},{float(mp_result):.16E},{scipy_result:.16E},{gnp_result:.16E}\n'
    
    with open('instances_beta_eq_zero.csv', 'a') as f:
        f.write(instance)

Second iteration

In [11]:
import pandas as pd

In [12]:
df_test = pd.read_csv('instances_beta_eq_zero.csv')

In [13]:
df_test

,x,alpha,mu,delta,mpmath,scipy,gnp
0,0.118216,27.116783,-2.158274,5.764235,9.999994e-01,9.9999940889103400E-01,9.9999940889102379E-01
1,4.504637,48.394205,-4.901985,43.929992,1.000000e+00,NAN,9.9999999999993516E-01
2,-3.558404,35.239417,-3.896193,14.967645,6.979935e-01,6.9799347240368237E-01,6.9799347240369103E-01
3,4.486494,27.261715,-3.687490,37.296710,1.000000e+00,NAN,9.9999999999816414E-01
4,-1.881685,23.184287,-0.677947,46.868379,1.985413e-01,NAN,NAN
...,...,...,...,...,...,...,...
995,0.654838,18.843563,-3.865584,25.968663,9.999383e-01,9.9993827933742463E-01,9.9993827933743118E-01
996,4.374292,14.346248,4.788995,14.979224,3.421822e-01,3.4218224649992784E-01,3.4218224649992751E-01
997,-4.812338,29.124041,0.421437,23.416039,3.229725e-09,3.2298373920161157E-09,3.2297416963089631E-09
998,3.656635,40.191090,-3.878165,22.050325,1.000000e+00,NAN,1.0000000000000275E+00


In [14]:
with open('instances_beta_eq_zero_algo.csv', 'a') as f:
    header = 'x,alpha,mu,delta,mpmath,scipy,gnp\n'
    f.write(header)


for i, row in df_test.iterrows():
    x, alpha, mu, delta, mp_result, scipy_result = row[['x', 'alpha', 'mu', 'delta', 'mpmath', 'scipy']]

    # GNP
    gnp_result = nig_cdf(x, alpha, 0, mu, delta)
    print(x, alpha, mu, delta, gnp_result)
    
    instance = f'{x},{alpha},{mu},{delta},{mp_result},{scipy_result},{gnp_result:.16E}\n'
    
    with open('instances_beta_eq_zero_algo.csv', 'a') as f:
        f.write(instance)

integration
23.6332
1 1
level 0 j = 1
level 1 j = 2
level 2 j = 4
level 3 j = 8
level 4 j = 16
level 5 j = 32
level 6 j = 63
level 7 j = 126
level 8 j = 250
level 9 j = 498
0.1182162470025671 27.116782747705887 -2.1582741889630075 5.764235016147439 0.999999408891028
asymptotic_xmu
4.504636963259353 48.394205038935645 -4.901985306618806 43.929991861661456 1.0
bessel_series
-3.5584038728036624 35.23941704343867 -3.8961933309651697 14.967644956238782 0.697993472403691
integration
9.30862
3 3
level 0 j = 1
level 1 j = 2
level 2 j = 4
level 3 j = 8
level 4 j = 16
level 5 j = 32
level 6 j = 63
level 7 j = 126
level 8 j = 250
level 9 j = 498
level 10 j = 993
4.486494471372438 27.26171517625945 -3.687489702693656 37.29670992519448 0.999999999998171
asymptotic_alpha
-1.8816854798951452 23.184287219731825 -0.6779473810531575 46.86837947038307 0.1985413190158847
integration
0.000301338
2 2
level 0 j = 1
level 1 j = 2
level 2 j = 4
level 3 j = 8
level 4 j = 16
level 5 j = 32
level 6 j = 63
level 7